In [ ]:
import numpy as np 
import pandas as pd 
import nltk
import json
import re
from nltk.sentiment import SentimentIntensityAnalyzer as sentiment

In [ ]:
train, samSub = pd.read_csv("../input/jigsaw-toxic-severity-rating/comments_to_score.csv"), \
    pd.read_csv("../input/jigsaw-toxic-severity-rating/sample_submission.csv")

with open("../input/english-contractions/contractions.json") as f:
    conts=json.load(f)
conts={k.lower(): v.lower() for k, v in conts.items()}

In [ ]:
emojis = ["\U0001F600", "\U0001F64F", "\U0001F300", "\U0001F5FF", "\U0001F680", "\U0001F6FF", "\U0001F1E0", "\U0001F1FF", "\U00002702", "\U000027B0", "\U000024C2", "\U0001F251"]
emojis

In [ ]:
stopWords = nltk.corpus.stopwords.words("english")

In [ ]:
token = nltk.tokenize.RegexpTokenizer(r"\w+")

In [ ]:
lemmatizer = nltk.stem.WordNetLemmatizer()

In [ ]:
train.loc[32, "text"]

In [ ]:
def cleaner(DF, c="text"):

    data=[]
    col=np.where(DF.columns == c)[0][0]
    
    for i, row in enumerate(DF.iterrows()):

        sentence=row[1][col]
        sentence=sentence.lower().split(" ")
        sentence=[word for word in sentence if word not in emojis]
        sentence=[word for word in sentence if 
                  "www" not in word and
                  "http" not in word and 
                  "https" not in word and 
                  "@" not in word]
        sentence=[word.replace(".", "") for word in sentence]
        sentence=[word.replace(",", "") for word in sentence]
        sentence=[word.replace("?", "") for word in sentence]
        sentence=[word.replace("!", "") for word in sentence]
        sentence=[conts[word] if word in conts else word for word in sentence] 
        sentence=" ".join(sentence)
        sentence=token.tokenize(sentence)
        sentence=[word for word in sentence if word not in stopWords]
        sentence=[lemmatizer.lemmatize(word) for word in sentence]
        sentence=[word.strip() for word in sentence]
        sentence=[word for word in sentence if not re.match(r"\S*\d+\S*", word)]
        sentence=[word for word in sentence if 
                  word != "rt" and 
                  word != "û_" and 
                  word != "amp" and 
                  word != "ûª" and
                  word != "ûªs" and
                  word != "ûò" and
                  word != "åè" and
                  word != "ìñ1"] 
        sentence=[re.sub(r"(.)\1{2,}\B", r"\1\1", word) for word in sentence]
        sentence=[re.sub(r"(.)\1{2,}\b", r"\1\1", word) for word in sentence]
        sentence=[word for word in sentence if len(word) > 0 and len(word) < 35]

        data.append(" ".join(sentence))
        
    return data

In [ ]:
tr = cleaner(train)
tr[32]

In [ ]:
tr[:4]

In [ ]:
s = sentiment()

In [ ]:
def score_rank(txt):
    score=[]
    for sentence in txt:
        score.append(s.polarity_scores(sentence)["neg"])
    rank=np.argsort(np.array(score))
    rank=np.argsort(rank)
    return rank

In [ ]:
samSub.score = score_rank(tr)

In [ ]:
samSub.to_csv("submission.csv", index=False)
samSub.head(n=10)